In [3]:
# implementing a stacking example 
# base estimators - LogisticRegression, KNN
# meta-estimator - Single DecisionTree

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.base import clone, BaseEstimator, TransformerMixin, ClassifierMixin

In [4]:
class StackingClassifier(BaseEstimator, TransformerMixin, ClassifierMixin):
    
    def __init__(self, classifiers):
        self.classifiers = classifiers
        self.meta_classifier = DecisionTreeClassifier()
    
    def fit(self, X, y):
        for clf in self.classifiers:
            clf.fit(X, y)
        
        self.meta_classifier.fit(self._get_meta_features(X), y)
        return self
    
    def _get_meta_features(self, X):
        probas = np.asarray([
            clf.predict_proba(X) for clf in self.classifiers
        ])
        return np.concatenate(probas, axis=1)
    
    def predict(self, X):
        return self.meta_classifier.predict(self._get_meta_features(X))
    
    def predict_proba(self, X):
        return self.meta_classifier.predict_proba(self._get_meta_features(X))

In [5]:
X, y = make_classification(
    n_samples=1000, n_features=50,n_informative=30,
    n_clusters_per_class=3, random_state=11
)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=11)

In [7]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
print('Logistic Regression Accuracy: ', lr.score(X_test, y_test))

Logistic Regression Accuracy:  0.816


In [8]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print('KNN Accuracy: ', knn.score(X_test, y_test))

KNN Accuracy:  0.836


In [9]:
base_classifiers = [lr, knn]
stacking = StackingClassifier(base_classifiers)
stacking.fit(X_train, y_train)
print('Stacking classifier accuracy: ', stacking.score(X_test, y_test))

Stacking classifier accuracy:  0.852
